## Coding Exercise #0703

### 1. Softmax regression (multi-class logistic regression):

In [ ]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

tf.disable_v2_behavior()

#### 1.1. Read in the data:

In [ ]:
# We will use Iris data.
# 4 explanatory variables.
# 3 classes for the response variable.
data_raw = load_iris()
data_raw.keys()

In [ ]:
# Print out the description.
# print(data_raw['DESCR'])

In [ ]:
X = data_raw['data']
y = data_raw['target']

In [ ]:
# Check the shape.
print(X.shape)
print(y.shape)

#### 1.2. Data pre-processing:

In [ ]:
# One-Hot-Encoding.
y = np.array(pd.get_dummies(y, drop_first=False))               # drop_frist = False for one-hot-encoding.
y.shape

In [ ]:
# Scaling
X = scale(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)
n_train_size = y_train.shape[0]

#### 1.3. Do the necessary definitions:

In [ ]:
batch_size = 100                                # Size of each (mini) batch.
n_epochs  = 30000                               # Number of epochs.
learn_rate = 0.05

In [ ]:
W = tf.Variable(tf.ones([4,3]))                 # Initial value of the weights = 1.
b = tf.Variable(tf.ones([3]))                   # Initial value of the bias = 1.

In [ ]:
X_ph = tf.placeholder(tf.float32, shape=(None, 4))    # Number of rows not specified. Number of columns = numbmer of X variables = 4.
y_ph = tf.placeholder(tf.float32, shape=(None,3))     # Number of rows not specified. Number of columns = number of classes of the y variable = 3.

In [ ]:
# Model.
# Not strictly necessary to apply the softmax activation. => in the end we will apply argmax() function to predict the label!
# y_model = tf.nn.softmax(tf.matmul(X_ph, W) + b)
# The following will work just fine.
y_model = tf.matmul(X_ph, W) + b

In [ ]:
loss  = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ph, logits=y_model))     # Loss = cross entropy. 

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learn_rate) 

In [ ]:
train = optimizer.minimize(loss)                                                                    # Define training.

In [ ]:
init = tf.global_variables_initializer()                                                            # Define Variable initialization.

#### 1.4. Training and Testing:

In [ ]:
with tf.Session() as sess:
        # Variables initialization.
        sess.run(init)
        # Training.
        for i in range(n_epochs):
            idx_rnd = np.random.choice(range(n_train_size),batch_size,replace=False)      # Random sampling w/o replacement for the batch indices.                
            batch_X, batch_y = [X_train[idx_rnd,:], y_train[idx_rnd,:]]                   # Get a batch.          
            my_feed = {X_ph:batch_X, y_ph:batch_y}                                        # Prepare the feed data as a dictionary.
            sess.run(train, feed_dict = my_feed)
            if (i + 1) % 2000 == 0: print("Step : {}".format(i + 1))                      # Print the step number at every multiple of 2000.
        # Testing.
        correct_predictions = tf.equal(tf.argmax(y_ph, axis=1), tf.argmax(y_model, axis=1))   # In argmax(), axis=1 means horizontal direction.
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))                   # Recast the Boolean as float32 first. Then calculate the mean.
        accuracy_value = sess.run(accuracy, feed_dict={X_ph:X_test, y_ph:y_test})             # Actually run the test with the test data.

Print the testing result.

In [ ]:
print("Accuracy = {:5.3f}".format(accuracy_value))